In [1]:
# 3rd Party Packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt
import numpy as np
# User Packages
from model import ShallowNet, MiniVGGNet
from callback import TrainingLogger

print('Tensorflow version: {}'.format(tf.__version__))

Tensorflow version: 1.12.0


In [2]:
def step_decay(epoch: int) -> float:
    initAlpha = 0.1
    factor = 0.5
    dropEvery = 5
    # Learning rate per epoch
    return float(initAlpha * (factor ** np.floor((1 + epoch) / dropEvery)))

In [3]:
print("[INFO] loading CIFAR-10 data...")
((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype(np.float) / 255.0
testX = testX.astype(np.float) / 255.0

# One-hot encoding
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

print('Training data shape: {}'.format(trainX.shape))
print('Training labels shape: {}'.format(trainY.shape))
print('Test data shape: {}'.format(testX.shape))
print('Test labels shape: {}'.format(testY.shape))

labelNames = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

print("[INFO] compiling model...")

callbacks = [
    LearningRateScheduler(step_decay),
    TrainingLogger(
        figure_path=r'output/minivggnet.png',
        json_path=r'output/minivggnet.json'
    )
]

opt = SGD(
    lr=0.01,
    decay=0.01/40,
    momentum=0.9,
    nesterov=True
)

parameters = {
    'width':    32,
    'height':   32,
    'channels': 3,
    'classes':  10
}
#model = ShallowNet.build(parameters)
model = MiniVGGNet.build(parameters)
model.compile(
    loss="categorical_crossentropy",
    optimizer=opt,
    metrics=["accuracy"]
)
# Print model summary
model.summary()

[INFO] loading CIFAR-10 data...


Training data shape: (50000, 32, 32, 3)
Training labels shape: (50000, 10)
Test data shape: (10000, 32, 32, 3)
Test labels shape: (10000, 10)
[INFO] compiling model...


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        9248      
__________

In [4]:
print("[INFO] training network...")
N_EPOCHS = 40
H = model.fit(
    trainX, trainY, 
    validation_data=(testX, testY),
    batch_size=32,
    epochs=N_EPOCHS,
    callbacks=callbacks,
    verbose=1
)

[INFO] training network...


Train on 50000 samples, validate on 10000 samples


Epoch 1/60


   32/50000 [..............................] - ETA: 17:53 - loss: 3.9399 - acc: 0.0938

  352/50000 [..............................] - ETA: 1:44 - loss: 5.4879 - acc: 0.1477 

...

47392/50000 [===========================>..] - ETA: 0s - loss: 1.1251 - acc: 0.5997

47648/50000 [===========================>..] - ETA: 0s - loss: 1.1252 - acc: 0.5997

47968/50000 [===========================>..] - ETA: 0s - loss: 1.1257 - acc: 0.5996

48256/50000 [===========================>..] - ETA: 0s - loss: 1.1254 - acc: 0.5996

KeyboardInterrupt: 

In [5]:
# Evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(
    testY.argmax(axis=1),
    predictions.argmax(axis=1),
    target_names=labelNames
))

print("TestX shape: {}".format(testX.shape))
print("Predictions shape: {}".format(predictions.shape))
print("TestY shape: {}".format(testY.shape))

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N_EPOCHS), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N_EPOCHS), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N_EPOCHS), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N_EPOCHS), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()

[INFO] evaluating network...


              precision    recall  f1-score   support

    airplane       0.70      0.62      0.66      1000
  automobile       0.79      0.78      0.79      1000
        bird       0.53      0.45      0.49      1000
         cat       0.47      0.37      0.41      1000
        deer       0.55      0.52      0.53      1000
         dog       0.54      0.56      0.55      1000
        frog       0.58      0.84      0.69      1000
       horse       0.70      0.67      0.69      1000
        ship       0.71      0.81      0.76      1000
       truck       0.76      0.73      0.75      1000

   micro avg       0.64      0.64      0.64     10000
   macro avg       0.63      0.64      0.63     10000
weighted avg       0.63      0.64      0.63     10000

TestX shape: (10000, 32, 32, 3)
Predictions shape: (10000, 10)
TestY shape: (10000, 10)


NameError: name 'H' is not defined

<Figure size 432x288 with 0 Axes>

In [8]:
import os
# Save the network to disk
print("[INFO] serializing network...")
#model_path = r'/home/share/models/cifar-10/shallownet.hdf5'
model_path = r'/home/share/models/cifar-10/minivggnet.hdf5'
directory, model_file = os.path.split(model_path)
if not os.path.exists(directory):
    os.makedirs(directory)
model.save(model_path)

[INFO] serializing network...
